# Observations and Insights

<hr>

In [1]:
# Dependencies and Setup (John)
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np

# CalFresh data file path (DeJuan)
calfresh_data_path = "Resources/CalFresh Data 19 - Current.csv"
# Census data file path (Jackson)
census_data_path = "Resources/co-est2019-alldata.csv"
# us-counties data file path (Siddharth)
all_counties_path = "Resources/us-counties.csv"


In [42]:
# Read the all three datasets (John)
calfresh_data = pd.read_csv(calfresh_data_path)



# rename columns.
calfresh_data.rename(columns = {'Calendar Year' : 'Year', 'Unemployment Monthly	' : 'Unemployment', 
                                'CalFresh Households' : 'Households', 'CalFresh Persons' : 'Persons', 
                                'EBT_FSP_dollars' : 'EBT Cash'}, inplace = True)

# Remove 'Date' Column from CSV, not usable for analysis. Is not an actual date, is only the Month and year joined. (DHall)
del calfresh_data['Date']

calfresh_df = pd.DataFrame(calfresh_data)
#calfresh_data.dtypes # use to find column data type.

# create dictionary to convert specific columns
##convert_dict = {'Unemployment Monthly' : float, 'Households' : int, 'Persons' : int, 'Applications Received' : int, 'Online Applications Received' : int, 'EBT Cash' : float}
##calfresh_data = calfresh_data.asytpe('convert_dict')
calfresh_df = calfresh_df.astype({'Households': 'int64'})

###, 'Unemployment Monthly': int,  
#                                    'Persons': 'int64', 'Applications Received': 'int64', 
#                                    'Online Applications Received': 'int64', 'EBT Cash': 'float64'
##calfresh_df['Year'] = calfresh_df['Year'].astype(int)  #single column dtype conversion


print(calfresh_df.dtypes)

#calfresh_data.head()

ValueError: invalid literal for int() with base 10: '1,906,041'

In [ ]:
#CA County Data (Jackson)
#Data cleanup: Separate date into 3 columns, MONTH, DATE, YEAR. Isolate counties to CA. Drop FIPS column. 
all_counties = pd.read_csv(all_counties_path,index_col=0, parse_dates=True)

#Drop FIPS column
all_counties_df = pd.DataFrame(all_counties).drop(["fips"], axis=1)

#Isolate counties to CA 
CA_counties_df = all_counties_df[all_counties_df["state"] == "California"]

CA_counties_df.head()

In [ ]:
#Separate date into three columns and rename (Jackson)
CA_counties_df.reset_index(inplace=True)

CA_counties_df["Day"] = CA_counties_df["date"].dt.day
CA_counties_df["Month"] = CA_counties_df["date"].dt.month
CA_counties_df["Year"] = CA_counties_df["date"].dt.year

CA_counties_df.rename(columns = {"date":"Date",
                                "county":"County",
                                "state":"State",
                                "cases":"Cases",
                                "deaths":"Deaths"}, inplace=True)
CA_counties_df

In [ ]:

#orange_county = pd.DataFrame(CA_counties_df).groupby()

In [ ]:
CA_counties_df.to_csv("Resources/CA_County_Data.csv")

In [ ]:
# Filter to show necessary columns adn rename columns to use initial Caps.
census_data = pd.read_csv(census_data_path)
census_data.rename(columns = {'STNAME' : 'State', 'CTYNAME' : 'County', 
                              'POPESTIMATE2019' : 'Population'}, inplace = True)
census_df = pd.DataFrame(census_data)
census_df.drop(census_df.columns[[0, 1, 2, 3, 4, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 
                                  22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 
                                  39, 40, 41, 42, 43, 44, 45, 46, 47]], axis = 1, inplace = True)

# filter rows in state that equal California and required columns (Siddharth)
census_ca = census_df.loc[census_df['State'] == 'California']
census_filtered = census_ca[['State', 'County', 'Population', 'DEATHS2019']]
census_filtered.head()

In [ ]:
#Filter calfreash data by 2020 and 2021 (DHall)
calfresh_data20_21 = calfresh_data[calfresh_data['Year']!=2019]

calfresh_data20_21

In [ ]:
#Export Updated CSV to Resource Folder (DHall)
calfresh_data20_21.to_csv('Resources/calfresh_data20_21.csv')

In [ ]:

# Combine CalFresh, census and counties data into a single dataset (JC)


# Create pd.DataFrame for population density by county. Timeframe 1-2020 to 3-2021

# Create pd.DataFrame for CalFresh recipients by county. Timeframe 1-2020 to 3-2021

# Create pd.DataFrame for total Covid cases by county. Timeframe 1-2020 to 3-2021
# Display the data table for preview


# Summary Statistics

## Line Chart 

In [ ]:
# Show CA state covid cases over months using line plot. Years on x-axis and population on y-axis
# Timeframe: Jan 2020 to March 2021


In [ ]:
# Graph trends amoungst various counties on a single plot. Pick 5, at least one urban area
# Timeframe: Jan 2020 to March 2021


In [ ]:
# Correlation Rise of Covid and CalFresh (maybe unemployment) vs number of people receiving CalFresh aid.

### Bar Chart

In [ ]:
# Number of Covid cases in all counties. Sort decending

In [ ]:
# Number of Covid cases in top 10 counties. Sort decending